In [1]:
import pandas as pd
from pyBiodatafuse import id_mapper
from pyBiodatafuse.analyzer.DREAMwalk.generate_embeddings import save_embedding_files
from pyBiodatafuse.analyzer.DREAMwalk.predict_associations import predict_dda
from pyBiodatafuse.analyzer.DREAMwalk.generate_similarity_net import save_sim_graph

from generate_files import generate_files

# Load file

In [2]:
gene_list = pd.read_csv("data/input_genes.csv")
gene_list.drop_duplicates(inplace=True)
gene_list.head(2)

,identifier
0,LOC729609
1,LOC105374060


In [4]:
bridgdb_df, bridgdb_metadata = id_mapper.bridgedb_xref(
    identifiers=gene_list,
    input_species="Human",
    input_datasource="HGNC",
    output_datasource=["NCBI Gene"],
)
bridgdb_df.head()

,identifier,identifier.source,target,target.source
0,DMP1,HGNC,1758,NCBI Gene
1,PNLIP,HGNC,5406,NCBI Gene
2,OR4N3P,HGNC,390539,NCBI Gene
3,SLC6A14,HGNC,11254,NCBI Gene
4,DEFB105A,HGNC,124906747,NCBI Gene


In [5]:
gene_list.shape, bridgdb_df.shape

((2023, 1), (1594, 4))

# TODO: Here add code from the common pipeline

In [2]:
kg_data = pd.read_csv("post-COVID_graph.csv")
kg_data.head(2)

,_id,_labels,Ensembl,diseaseID,drugID,gene_count,id,name,ncbiID,source,subcellular_loc,_start,_end,_type,score,source.1,type,atcClassification,class,uniprotID
0,13440.0,:Protein,ENSG00000166535,NaN,NaN,NaN,ENSG00000166535,A2ML1,144568.0,BridgeDB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,13441.0,:Disease,NaN,C0028326,NaN,NaN,C0028326,Noonan Syndrome,NaN,DisGeNET,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
generate_files(kg_data)

Graph file is saved!
Node types file is saved!
Hierarchy file is saved!
Drug-Disease association file is saved!
Drug-Disease association file is saved!
Drug-Disease association file is saved!
Drug-Disease association file is saved!
Drug-Disease association file is saved!
Drug-Disease association file is saved!
Drug-Disease association file is saved!
Drug-Disease association file is saved!
Drug-Disease association file is saved!
Drug-Disease association file is saved!


In [ ]:
dis_gen.save_dis_sim("post-COVID_graph.csv", "dis_sim.csv")

In [ ]:
networkf = "graph.txt"
hierf = "hierarchy.csv"
simf = "similarty_graph_drugs.tsv"
cutoff = 0.4

In [ ]:
save_sim_graph(networkf=networkf, hierf=hierf, outputf=simf, cutoff=cutoff)

In [ ]:
# create similarity graph merging disease and drugs
dis_sim = pd.read_csv("dis_sim.tsv", sep="\t", header=None)
drug_sim = pd.read_csv("similarty_graph_drugs.tsv", sep="\t", header=None)

In [ ]:
similarty_graph = pd.concat([dis_sim, drug_sim], ignore_index=True)

similarty_graph[4] = range(0, len(similarty_graph))
similarty_graph.to_csv("similarty_graph.txt", sep="\t", index=False, header=False)


# Generate node embeddings by teleport-guided randomwalk

nodetypef = "nodetypes.tsv"
embeddingf = "embedding_file.pkl"
simf = "similarty_graph.txt"

save_embedding_files(
    netf=networkf, sim_netf=hierf, outputf=embeddingf, nodetypef=nodetypef, tp_factor=0.3
)


# Predict drug-disease association

pairf = "dda_files/dda10.tsv"
modelf = "results/clf10.pkl"
embeddingf = "embedding_file.pkl"

predict_dda(embeddingf=embeddingf, pairf=pairf, modelf=modelf)

# Calculate drug scores, run only this section to just check the results (running the algorithm is quite time consuming, especially the embedding file)
model_folder = "results"
query_disease = "C0000000"  # Post COVID 19 node ID
kgfile = "preprocessed_graph.csv"
find_candidates(kgfile, embeddingf, model_folder, query_disease, candidates_count=20)